# Privacy Preserving Machine Learning

Course taught by Aurélien Bellet

Course page: http://researchers.lille.inria.fr/abellet/teaching/private_machine_learning_course.html

# Practical session 2: Differential privacy for non-numeric queries & using composition results

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC

## Dataset

We will be working again with the US Census dataset. You can read about the dataset [here](https://archive.ics.uci.edu/ml/datasets/census+income).

The following line loads the dataset from [OpenML](https://www.openml.org/) with the `fetch_openml` method of `sklearn`. The option `as_frame=True` loads the dataset in `pandas DataFrame` format: this keeps the attributes in their original form and will be more convenient to work with. If you prefer working with a numpy array (not recommended), set `as_frame=False`.

In [2]:
dataset_handle = fetch_openml(name='adult', version=2, as_frame=True)
dataset = dataset_handle.frame

In [3]:
n, d = dataset.shape
print(n, d)
dataset.head(10)

48842 15


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
5,34.0,Private,198693.0,10th,6.0,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,30.0,United-States,<=50K
6,29.0,NaN,227026.0,HS-grad,9.0,Never-married,NaN,Unmarried,Black,Male,0.0,0.0,40.0,United-States,<=50K
7,63.0,Self-emp-not-inc,104626.0,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103.0,0.0,32.0,United-States,>50K
8,24.0,Private,369667.0,Some-college,10.0,Never-married,Other-service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
9,55.0,Private,104996.0,7th-8th,4.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,10.0,United-States,<=50K


## Question 1 ("most common" query)

Implement a function for "most common" queries: it takes as input a dataset (`DataFrame`), an attribute (e.g., `"education"`), and returns the most common value of this attribute in the dataset (ties can be broken arbitrarily). Note that in general, the output of such queries is non-numeric. The pandas methods `value_counts`, `idxmax()` or `mode` can be useful here.

Test your function on the dataset.

In [4]:
def most_common_query(df, attribute):
    '''
    Parameters
    ----------
    df : DataFrame
        Dataset
    attribute : string
        Name of an attribute (with categorical values)
        
    Returns
    -------
    value : float or string
        The most common value of `attribute` in dataset `df`
    '''
    
    # TO COMPLETE
    pass

## Question 2 (Exponential mechanism)

Implement the exponential mechanism, i.e., a function which takes as input the list of possible outputs, the corresponding scores (assuming a numpy array format will be most convenient), the sensitivity of the score function, the desired value of $\epsilon$ and a random seed (for reproducibility), and returns an output selected in an $\epsilon$-differentially private way. To sample an ouput, use `numpy.random.choice`.

**Careful**: if you implement the exponential mechanism naively you will get numerical errors because un-normalized probabilities will quickly become very large. You should compute everything in log domain and only take the exponential at the end. To compute the normalization term in log domain, you can use `scipy.special.logsumexp`.

In [5]:
def exponential_mechanism(outputs, scores, sensitivity, eps, random_state=None):
    '''
    Parameters
    ----------
    outputs : list
        Possible outputs for the query
    scores : array of float 
        Scores associated to each output
    sensitivity : float
        The sensitivity of the score
    eps : float
        Parameter epsilon of differential privacy
    random_state : int, optional (default=None)
        Random seed
        
    Returns
    -------
    private_q : float or string (sans as outputs)
        An eps-DP evaluation of the query
    '''
    
    rng = np.random.RandomState(random_state)
    # TO COMPLETE

## Question 3 (Exponential mechanism on "most common" queries)

We would like to use the exponential mechanism to privately answer the following queries:
- What is the most common level of education of people in the dataset?
- What is the most common occupation of people in the dataset?
- What is the most common age of people in the dataset?

Define an appropriate score function for these queries. What is the sensitivity of your score function?

Use the exponential mechanism to privately answer these queries for $\epsilon=0.1$ and several random executions. Compare the output to the non-private result, and explain why for some queries the true (non-private) output is returned more often.

In [6]:
def private_most_common_query(df, attribute, eps, random_state=None):
    '''
    Parameters
    ----------
    df : DataFrame
        Dataset
    attribute : string
        Name of an attribute (with categorical values)
    eps : float
        Parameter epsilon of differential privacy
    random_state : int, optional (default=None)
        Random seed
        
    Returns
    -------
    private_q : float or string
        An eps-DP evaluation of the "most common" query on `attribute`
    '''
    
    d = dict(df[attribute].value_counts())
    outputs = list(d.keys())
    # TO COMPLETE

In [7]:
query_attribute = ['education', 'occupation', 'age']

for attr in query_attribute:
    print("\nAttribute queries:", attr)
    for r in range(20):
        print(private_most_common_query(dataset, attr, 0.1))


Attribute queries: education
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None

Attribute queries: occupation
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None

Attribute queries: age
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


## Question 4 (Exponential mechanism for model selection)

Suppose that you would like to build a model to predict the income of an individual based on its personal features (for instance, to study which features are predictive of income). You have access to a small *public* dataset of 50 people in the same format as the US Census dataset we have been working with so far, and you would like to train a classifier to predict whether an individual makes more than 50$/year from its personal features. However, there are hyper-parameters to tune and you know the dataset is too small to reliably estimate the generalization performance of each model to select the best.

There exists a larger dataset held by the US Census on which you could perform model selection, but this dataset is *private*. Therefore you would like to privately perform model selection on this dataset, i.e., among the set of models you trained on the small public dataset, you would like to select the one which performs best on the private dataset in an $\epsilon$-DP fashion.

To simulate this scenario, we use the same dataset as before. For convenience, we load it in numpy format where all categorical features have been coded as numerical variables, and remove the features that have missing values. We then split the full dataset into a small dataset with 50 people (the *public* dataset) and a larger dataset of 48K+ people (the *private* dataset).

In [8]:
X, y = fetch_openml(name='adult', version=2, return_X_y=True, as_frame=False)
features_complete = (np.isnan(X).sum(axis=0)) == 0 # features without missing values
X = X[:, features_complete]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_public, X_private, y_public, y_private = train_test_split(X, y, train_size=50, random_state=42)
print(X_public.shape, y_public.shape, X_private.shape, y_private.shape)

(50, 11) (50,) (48792, 11) (48792,)


1. What is the output space of the query?
2. How to define the score function in this case? What is its sensitivity?
3. Implement a function for private model selection with the exponential mechanism: it takes as input the public and the private datasets, a scikit-learn model (e.g., `LogisticRegression()`), a `ParameterGrid` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html)) giving the possible hyper-parameters and values, the desired value of $\epsilon$ and a random seed (for reproducibility), and returns a set of hyper-parameters selected in an $\epsilon$-DP way.
4. Run this to tune the value of the parameter `C` of `LogisticRegression()`. You may also try other cases, e.g., tuning the choice of kernel, parameter `C` and parameter `gamma` of the RBF kernel in a SVM model.

In [9]:
def private_model_selection(X_public, y_public, X_private, y_private, model, grid, eps, random_state=None):
    '''
    Parameters
    ----------
    X_public : array of shape (n1, d)
        Public dataset
    y_public : array of shape (n1,)
        Public labels
    X_private : array of shape (n2, d)
        Private dataset
    y_private : array of shape (n2,)
        Private labels
    model : BaseEstimator
        An instance of a scikit-learn model
    grid : ParameterGrid
        A grid describing the set of hyperparameters to tune and their possible values
    eps : float
        Parameter epsilon of differential privacy
    random_state : int, optional (default=None)
        Random seed
        
    Returns
    -------
    private_best_params : dict
        An eps-DP evaluation of best hyperparameter search
    '''

    n_private = X_private.shape[0]
    accuracies = np.zeros(len(grid))
    for i, param in enumerate(grid):
        model.set_params(**param)
        # TO COMPLETE
    

In [10]:
params = {'C':np.geomspace(1e-3, 1e3, num=100)}
grid = ParameterGrid(params)
model = LogisticRegression(solver='liblinear')
# TO COMPLETE

In [11]:
params = [{'kernel':['rbf'], 'gamma':np.geomspace(1e-4, 1e4, num=20), 'C': np.geomspace(1e-3, 1e3, num=10)},
              {'kernel':['linear'], 'C':np.geomspace(1e-3, 1e3, num=10)}]
grid = ParameterGrid(params)
model = SVC()
# TO COMPLETE

## Question 5 (using composition results)

For this question we will work with the MovieLens 1M dataset, which contains 1,000,000 ratings in {1, 2, 3, 4, 5} given by 6040 users on 3706 movies (there is of course a lot of missing values!). There exists even bigger versions of this dataset, see [here](https://grouplens.org/datasets/movielens/).

The code below loads the MovieLens 1M dataset, formats it to have users as rows and drops movies with less than 1000 ratings. Before executing this, [download the dataset](http://files.grouplens.org/datasets/movielens/ml-1m.zip) and unzip the file in the same directory as your Jupyter notebook.

In [12]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-1m/users.dat', sep='::', names=u_cols, engine='python')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, engine='python')

m_cols = ['movie_id', 'title', 'release_date']
movies = pd.read_csv('ml-1m/movies.dat', sep='::', names=m_cols, engine='python', encoding_errors='ignore')

movie_ratings = pd.merge(movies, ratings)
user_ratings = pd.merge(users, movie_ratings[['user_id', 'title', 'rating']])
user_ratings = (user_ratings.pivot_table(index=['user_id', 'age', 'sex', 'occupation', 'zip_code'],
                                         columns='title', values='rating').reset_index())
user_ratings.columns.name=""
user_ratings = user_ratings.dropna(thresh=1000, axis=1)
movie_list = list(user_ratings.columns[5:])
n_movies = len(movie_list)
user_ratings

,user_id,age,sex,occupation,zip_code,2001: A Space Odyssey (1968),"Abyss, The (1989)","African Queen, The (1951)",Air Force One (1997),Airplane! (1980),...,"Untouchables, The (1987)","Usual Suspects, The (1995)",Wayne's World (1992),When Harry Met Sally... (1989),Who Framed Roger Rabbit? (1988),Willy Wonka and the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)",X-Men (2000),Young Frankenstein (1974)
0,1,F,1,10,48067,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
1,2,M,56,16,70072,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,M,25,15,55117,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,M,45,7,02460,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,M,25,20,55455,NaN,1.0,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,4.0,NaN,NaN,4.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,F,25,15,32603,5.0,4.0,4.0,NaN,3.0,...,NaN,3.0,NaN,2.0,2.0,NaN,4.0,NaN,NaN,4.0
6036,6037,F,45,1,76006,5.0,4.0,NaN,NaN,NaN,...,NaN,4.0,NaN,4.0,NaN,NaN,4.0,4.0,NaN,NaN
6037,6038,F,56,1,14706,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,6039,F,45,0,01060,4.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN


We would now like to privately compute the majority rating among users for each movie in the dataset. This is a lot of queries (207) and unfortunately each user can potentially affect all of them. Compare the use of simple and advanced composition to calibrate the noise for each query so as to satisfy a fixed $(\epsilon,\delta)$-DP guarantee for the entire process. Comment the results.

**Note:** To simplify a bit the implementation, you can:
- assume that all possible ratings (1, 2, 3, 4, 5) have non-zero count for each movie (this is likely to be the case since we are working on movies with at least 1,000 ratings). A better implementation would make sure that ratings not seen for a movie are assigned a count of 0 so that the exponential mechanism is truly safe (see lecture).
- measure the error as the frequency of getting the same answer as the non-private query, that is, you can ignore ties (several ratings may have maximal count). Again, ties are unlikely to happen for movies with more than 1,000 ratings.

In [13]:
def compute_error(df, attribute_list, eps, random_state=None):
    '''
    Parameters
    ----------
    df : DataFrame
        Dataset
    attribute_list : list of string
        List of attribute names
    eps : float
        Parameter epsilon of differential privacy
    random_state : int, optional (default=None)
        Random seed
        
    Returns
    -------
    error : float
        The frequency that the private query returns the same answer as the non-private query
    '''
    
    error = 0
    for a in attribute_list:
        # TO COMPLETE
        pass
    error /= len(attribute_list)
    return error

In [14]:
eps_final = 2.
k = len(movie_list) # number of movies

# TO COMPLETE
eps0_simple = 0
eps0_advanced = 0 

n_runs = 5
error_simple = 0
error_advanced = 0
for r in range(n_runs):
    # TO COMPLETE
    # error_simple += 
    # error_advanced += 
    pass
error_simple /= n_runs
error_advanced /= n_runs

print("**Simple composition**")
print("Value of epsilon for each individual query", eps0_simple)
print("Average error:", error_simple)

print("\n**Advanced composition**")
print("Value of epsilon for each individual query", eps0_advanced)
print("Average error:", error_advanced)

**Simple composition**
Value of epsilon for each individual query 0
Average error: 0.0

**Advanced composition**
Value of epsilon for each individual query 0
Average error: 0.0


## Bonus Question 1 (Laplace mechanism and Report Noisy Max)

1. Propose a simple way to privately answer the queries of Questions 3 and 4 using the Laplace mechanism instead of the exponential mechanism. What is the $\ell_1$ sensitivity for the two cases? For Question 4, do you think it is possible to do better with the Laplace mechanism?
2. Read about Report Noisy Max in Section 3.3 of [Dwork & Roth (2014)](https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf).

## Bonus Question 2 (Even better composition)

In Question 5 above, when using advanced composition, we have picked the $\epsilon_0$ to use for each individual query using the convenient cororally seen in the lecture. However this corollary is not tight and it is better to use the advanced composition theorem directly. Implement a function to choose the smallest admissible value $\epsilon_0$ according to the theorem and evaluate the gains. To get even slightly better composition in some regimes (and also more flexible), use the results of [Kairouz et al. (2015)](http://proceedings.mlr.press/v37/kairouz15.pdf), for instance Corollary 4.1 therein.